# Induced gap and magnetic field

# TO-DO:
* Tune induced gap algo

In [ ]:
# 1. Standard library imports
import os.path

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from toolz import partition_all

# 3. Internal imports
import funcs
from funcs import constants

hv.notebook_extension()
print(kwant.__version__)
import warnings
# warnings.filterwarnings('ignore', r'Interpreting')

# Usage

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=False,
              mu=2, mu_lead=2, c_tunnel=5/8, V=0, V_barrier=0, **constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=50, phi=135, r1=50, r2=70, shape='circle',
                 with_leads=True, with_shell=True)

syst = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst);

In [ ]:
lead = syst.leads[0]

def get_cross_section(syst, pos, direction):
    coord = np.array([s.pos for s in syst.sites if s.pos[direction] == pos])
    cross_section = np.delete(coord, direction, 1)
    return cross_section

def get_densities(lead, k, params):

    xy = get_cross_section(lead, pos=0, direction=0)
    h, t = lead.cell_hamiltonian(params=params), lead.inter_cell_hopping(params=params)
    h_k = h + t * np.exp(1j * k) + t.T.conj() * np.exp(-1j * k)

    vals, vecs = np.linalg.eigh(h_k)
    indxs = np.argsort(np.abs(vals))
    vecs = vecs[:, indxs]
    vals = vals[indxs]
    
    norbs = funcs.lat_from_syst(lead).norbs
    densities = np.linalg.norm(vecs.reshape(-1, norbs, len(vecs)), axis=1)**2
    return xy, vals, densities.T

xy, energies, densities = get_densities(lead, 0.1, params)
wfs = [kwant.plotter.mask_interpolate(xy, density, oversampling=1)[0] for density in densities[:40]]

ims = {E: hv.Image(wf) for E, wf in zip(energies, wfs)}
hv.HoloMap(ims, kdims=[hv.Dimension('E', unit='meV')])

In [ ]:
%%opts Image [colorbar=True]

x = 20
xy = get_cross_section(syst, x, 0)

V = lambda x, y, z: 10 * z / syst_pars['r1']

potential = np.array([V(*s.pos) for s in syst.sites if s.pos[0]==x])

hv.Image(np.rot90(kwant.plotter.mask_interpolate(xy, potential, oversampling=1)[0]))

In [ ]:
%%opts Path [aspect='square']

lead = funcs.make_lead(10, 50, 70, 135, 0, True, 'circle').finalized()
params['B_x'] = 1
params['B_y'] = 0
params['orbital'] = True
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

# params['orbital'] = False
params['c_tunnel'] = 5/8
Es1 = funcs.bands(lead, params, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2 + (p1*p2)[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print('The bandgap is {} meV'.format(gap))

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, c_tunnel=5/8, V=0, V_barrier=1, **constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params)

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    import hpc05
    client = hpc05.Client(profile='pbs3', timeout=60, extra_args='--timeout=1200')
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client()
    print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/200px-3D_Spherical.svg.png)

In [ ]:
params = dict(Delta=60, V=0, c_tunnel=5/8, **constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False,
                 L=2000, phi=135, r1=50, r2=70, shape='circle',
                 with_leads=True, with_shell=True)

def cond(val, syst_pars=syst_pars, params=params):
    import funcs
    val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
    params = dict(**params, **val)
    params['mu_lead'] = val['mu']
    
    for x in ['angle']:
        syst_pars[x] = params.pop(x)

    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **params)


Bs = funcs.spherical_coords_vec(rs=np.linspace(0, 2, 51),
                                thetas=[0, 90],
                                phis=[0, 90])

vals = funcs.named_product(B=Bs,
                           V_bias=np.linspace(-0.25, 0.25, 51),
                           V_barrier=[15],
                           g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           mu=[5, 10, 15, 20],
                           angle=[0])
print(len(vals))

for i, chunk in enumerate(partition_all(20000, vals)):
    fname = 'conductance_{:03d}_sc_correction.hdf'.format(i)
    if not os.path.exists(fname):
        G = lview.map_async(cond, chunk)
        G.wait_interactive()
        G = G.result()

        df = pd.DataFrame(G)

        df = df.assign(**syst_pars)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

In [ ]:
# syst_pars = dict(a=10, angle=45, phi=135, r1=50, r2=70, shape='circle', with_shell=True)

# def gap(val, syst_pars=syst_pars, params=params):
#     import funcs
#     val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
#     params = dict(**params, **val)
#     lead = funcs.make_lead(**syst_pars)
#     return dict(E_gap=funcs.find_gap(lead, params), **val)

# for i, chunk in enumerate(partition_all(3000, vals)):
#     fname = 'gaps_{:03d}.hdf'.format(i)
#     if not os.path.exists(fname):
#         E_gap = lview.map_async(gap, chunk)
#         E_gap.wait_interactive()
#         E_gap = E_gap.result()

#         df = pd.DataFrame(E_gap)
        
#         df = df.assign(**syst_pars).assign(**params).assign(**constants.__dict__)
#         df = df.assign(git_hash=funcs.get_git_revision_hash())
#         df.to_hdf(fname, 'all_data', mode='w')

In [ ]:
# from glob import glob
# df = pd.concat([pd.read_hdf(f) for f in glob("G_*")], axis=0)
# df = pd.read_hdf('G_0_.hdf')